<a href="https://colab.research.google.com/github/HenriqueCSilva/Magnificent-Seven-Support-Prices-Analysis/blob/main/Magnificent_Seven_BDR_Support_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# magnificent_seven_bdr_support.py
#
# Description:
# This script fetches historical data for the Brazilian Depositary Receipts (BDRs)
# of the "Magnificent Seven" tech stocks (Apple, Microsoft, Alphabet, Amazon,
# Nvidia, Tesla, and Meta). It then calculates and displays potential support
# price levels based on recent historical lows.
#
# A support level in stock trading is a price point where a downtrend is
# expected to pause due to a concentration of demand. This script identifies
# potential support by finding the lowest closing prices over the last 3, 6,
# and 12 months.

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_support_levels(ticker):
    """
    Downloads historical data for a given BDR ticker and calculates
    potential support levels.

    Args:
        ticker (str): The stock ticker symbol for the BDR (e.g., 'AAPL34.SA').

    Returns:
        dict: A dictionary containing the ticker and its calculated support
              levels for 3-month, 6-month, and 12-month periods. Returns None
              if data cannot be fetched.
    """
    try:
        # Download the last 1 year of historical data
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)

        # Set auto_adjust=True to silence the warning and get dividend-adjusted prices
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)

        # yfinance can sometimes return DataFrames with duplicate columns.
        # This causes errors in calculations, so we remove any duplicates.
        stock_data = stock_data.loc[:,~stock_data.columns.duplicated()]

        if stock_data.empty:
            print(f"Warning: No data found for {ticker}. It might be delisted or the ticker is incorrect.")
            return None

        # Define time periods for analysis
        three_months_ago = end_date - timedelta(days=90)
        six_months_ago = end_date - timedelta(days=180)

        # Filter data for the last 3 and 6 months
        data_3m = stock_data[stock_data.index >= three_months_ago]
        data_6m = stock_data[stock_data.index >= six_months_ago]

        # Calculate the lowest closing price in each period
        support_12m = stock_data['Low'].min()
        support_6m = data_6m['Low'].min()
        support_3m = data_3m['Low'].min()

        current_price = stock_data['Close'].iloc[-1]

        # yfinance can return a multi-index DataFrame, causing the results above
        # to be a Series instead of a float. We extract the value if it's a Series.
        if isinstance(current_price, pd.Series):
            current_price = current_price.iloc[0]
        if isinstance(support_3m, pd.Series):
            support_3m = support_3m.iloc[0]
        if isinstance(support_6m, pd.Series):
            support_6m = support_6m.iloc[0]
        if isinstance(support_12m, pd.Series):
            support_12m = support_12m.iloc[0]

        return {
            "ticker": ticker,
            "current_price": f"{current_price:.2f}",
            "support_3m": f"{support_3m:.2f}",
            "support_6m": f"{support_6m:.2f}",
            "support_12m": f"{support_12m:.2f}",
        }
    except Exception as e:
        print(f"An error occurred while processing {ticker}: {e}")
        return None

def main():
    """
    Main function to define BDRs, fetch their support levels,
    and print a summary table.
    """
    # Dictionary of the Magnificent Seven and their BDR tickers on B3
    # Note: We append '.SA' for Yahoo Finance to recognize Brazilian stocks
    magnificent_seven_bdrs = {
        "Apple": "AAPL34.SA",
        "Microsoft": "MSFT34.SA",
        "Alphabet (Google)": "GOGL34.SA",
        "Amazon": "AMZO34.SA",
        "NVIDIA": "NVDC34.SA",
        "Tesla": "TSLA34.SA",
        "Meta Platforms": "M1TA34.SA",
    }

    print("Fetching Support Prices for Magnificent Seven BDRs...")
    print("-" * 70)

    results = []
    for company, ticker in magnificent_seven_bdrs.items():
        print(f"Processing {company} ({ticker})...")
        levels = get_support_levels(ticker)
        if levels:
            levels['company'] = company
            results.append(levels)

    if not results:
        print("\nCould not retrieve data for any of the specified BDRs.")
        return

    # Display the results in a formatted table
    print("\n" + "="*70)
    print("Summary of Potential Support Levels (in BRL)")
    print("="*70)
    print(f"{'Company':<20} | {'Ticker':<12} | {'Current':<10} | {'3-Month':<10} | {'6-Month':<10} | {'12-Month':<10}")
    print("-" * 70)

    for res in results:
        print(f"{res['company']:<20} | {res['ticker']:<12} | {res['current_price']:<10} | {res['support_3m']:<10} | {res['support_6m']:<10} | {res['support_12m']:<10}")

    print("-" * 70)
    print("\nDisclaimer: This analysis is for informational purposes only and not financial advice.")


if __name__ == "__main__":
    main()

Fetching Support Prices for Magnificent Seven BDRs...
----------------------------------------------------------------------
Processing Apple (AAPL34.SA)...
Processing Microsoft (MSFT34.SA)...
Processing Alphabet (Google) (GOGL34.SA)...
Processing Amazon (AMZO34.SA)...
Processing NVIDIA (NVDC34.SA)...
Processing Tesla (TSLA34.SA)...
Processing Meta Platforms (M1TA34.SA)...

Summary of Potential Support Levels (in BRL)
Company              | Ticker       | Current    | 3-Month    | 6-Month    | 12-Month  
----------------------------------------------------------------------
Apple                | AAPL34.SA    | 61.30      | 0.00       | 0.00       | 0.00      
Microsoft            | MSFT34.SA    | 113.00     | 0.00       | 0.00       | 0.00      
Alphabet (Google)    | GOGL34.SA    | 107.67     | 0.00       | 0.00       | 0.00      
Amazon               | AMZO34.SA    | 62.40      | 0.00       | 0.00       | 0.00      
NVIDIA               | NVDC34.SA    | 20.00      | 0.00       | 0.0